In [1]:
import pandas as pd
import numpy as np

In [2]:
path = '/data/users/kgruber/other-data/impacts_paper/'
results_path = path + 'individual/'

read main results file

In [3]:
columns = ['Timestamp','1a. SCOPUS ID','1b. DOI link','1c. Last name of the first author','1d. Year of publication:','2. VRES technology:','3. Metrics used:','4a. Numeric value, e.g., 0.3, 1.5, 3','4b. Is power-related component of the land-use requirement expressed as energy e.g., ha/GWh/year?','4c-1. Measurement unit:','4c-2. Measurement unit:','4d. Type of value','4e. Geographical scale:','5a.  Temporary needed areas for the plant construction are included ','5b. Main modules of the plant, e.g., wind turbines,  solar PV, heliostats, etc. are included:','5c. Roads are included:','5d. The substation is included:','5e. Transmission lines are included:','5f. Other infrastructure included: ','6a. Area occupied by the infrastructure (questions 5a-f) was:','6b. Were the following data sources used to measure the area occupied by the infrastructure:','7. Area of the infrastructure, which is not located nearby the facility but necessary for its operation (e.g., mines, plants that manufacture PV-modules etc.),  is reported:','8a. Is spacing area included?','8b. Spacing area was quantified by using:','9a. Project area is reported:','9b. Project area refer to ','9c. Data on project were derived from','10. Power-related component of land-use requirements  is represented by: ','11a. Installation year(s):','11b. Area is estimated per: ','11c. Power / energy is estimated per: ','Additional characteristics reported:','Characterstic-1','Value-1','Unit-1','More characteristics are reported:','Characterstic-2','Value-2','Unit-2','More characteristics are reported:','Characterstic-3','Value-3','Unit-3','More characteristics are reported:','Characterstic-4','Value-4','Unit-4','More characteristics are reported:','Characterstic-5','Value-5','Unit-5','More characteristics are reported:','Characterstic-6','Value-6','Unit-6','More characteristics are reported:','Characterstic-7','Value-7','Unit-7','More characteristics are reported:','Characterstic-8','Value-8','Unit-8','More characteristics are reported:','Characterstic-9','Value-9','Unit-9','More characteristics are reported:','Characterstic-10','Value-10','Unit-10','More characteristics are reported:','Reviewed by','Comments','Country:','Subregion, e.g., state, biome, county, etc.:','','Any additional information about the location of the facility:','Provide the reference to the place in the paper, where the value can be found, e.g., table X, figure X, page XX paragraph X']

In [4]:
results = pd.read_csv(path + 'lit-review (Responses).csv').iloc[:,:79]
results.columns = results.columns.str.replace('  ',' ').str.strip()

read additional results files and merge

In [5]:
additional_files = ['Ciliberti_Johannes.xlsx','DeMeo_1997.xlsx','Lingzhi_2018.xlsx','MacKey_2013.xlsx',
                    'Olga_Enevoldsen2021.xlsx','Reddy_2012.xlsx']
multitab_file = 'additional-values-for-review-johannes.xlsx'
onetab_file = 'Pletk_2007.xlsx'
sheets = pd.ExcelFile(path + multitab_file, engine='openpyxl').sheet_names
sheet = 'lines_256_266'

In [6]:
for additional_file in additional_files:
    f1 = pd.read_excel(path + additional_file,engine='openpyxl').iloc[:,:79]
    f1.columns = list(f1.columns[:-1].str.replace('  ',' ').str.strip()) + [results.columns[-1]]
    #f1.columns = results.columns
    results = pd.concat([results,f1],axis=0)

In [7]:
f2 = pd.read_excel(path + onetab_file,engine='openpyxl',sheet_name=sheet).iloc[:,:79]
#f2.columns = list(f2.columns[:-1].str.replace('  ',' ').str.strip()) + [results.columns[-1]]
f2.columns = results.columns
results = pd.concat([results,f2],axis=0)

In [8]:
for additional_sheet in sheets:
    if additional_sheet=='Capellan-Perez':
        f2 = pd.read_excel(path + multitab_file, engine='openpyxl',sheet_name=additional_sheet,header=None).iloc[:,:79]
    else:
        f2 = pd.read_excel(path + multitab_file, engine='openpyxl',sheet_name=additional_sheet).iloc[:,:79]
    f2.columns = results.columns
    results = pd.concat([results,f2],axis=0)

remove duplicates

In [9]:
results_no_duplicates = results.drop_duplicates()
results_no_duplicates.index = range(len(results_no_duplicates))

create new columns

In [10]:
ID = pd.DataFrame({'ID':np.arange(len(results_no_duplicates))})
DATA_SOURCE = pd.DataFrame({'1e. Data source':None},index=range(len(results_no_duplicates)))
PV_AC_DC = pd.DataFrame({'2b. Is PV AC or DC:':None},index=range(len(results_no_duplicates)))
VALUE_CALC_FORMULA = pd.DataFrame({'4f. Is the value original or derived:':None,
                                   '4g. Formula for calculation:':None,
                                   '4h. File for calculations:':None},index=range(len(results_no_duplicates)))

CHECKED = pd.DataFrame({'Checked':None},index=range(len(results_no_duplicates)))

In [11]:
results_no_duplicates_new_columns = pd.concat([ID,results_no_duplicates.iloc[:,:5],DATA_SOURCE,
                                               results_no_duplicates.iloc[:,5:6],PV_AC_DC,
                                               results_no_duplicates.iloc[:,6:13],VALUE_CALC_FORMULA,
                                               results_no_duplicates.iloc[:,13:],CHECKED],axis=1)

fill in missing names

In [12]:
results_no_duplicates_new_columns['Reviewed by'] = results_no_duplicates_new_columns['Reviewed by'].fillna('Missing_name')

split in single files

In [13]:
for name in results_no_duplicates_new_columns['Reviewed by'].unique():
    results_name = results_no_duplicates_new_columns[results_no_duplicates_new_columns['Reviewed by']==name]
    results_name.to_excel(results_path + 'results_' + name + '.xlsx')